# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: Unveiling the Power of DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI, a game-changer in the tech industry. Developed by renowned expert Andrew Ng, this platform offers top-notch courses on deep learning, neural networks, and more. Whether you're a beginner or an advanced learner, DeepLearning.AI caters to all skill levels, making complex concepts easy to grasp. Gain practical knowledge, work on real-world projects, and join a thriving community of AI enthusiasts. Elevate your skills, boost your career, and unlock endless possibilities in AI. Embrace the future of technology with DeepLearning.AI today.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI, a game-changer in the tech industry. Developed by renowned expert Andrew Ng, this platform offers top-notch courses on deep learning, neural networks, and more. Whether you're a beginner or an advanced learner, DeepLearning.AI caters to all skill levels, making complex concepts easy to grasp. Gain practical knowledge, work on real-world projects, and join a thriving community of AI enthusiasts. Elevate your skills, boost your career, and unlock endless possibilities in AI. Embrace the future of technology with DeepLearning.AI today.

--------------------------------------------------------------------------------
Cri

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI, a game-changer in the tech industry. Developed by renowned expert Andrew Ng, this platform offers top-notch courses on deep learning, neural networks, and more. Whether you're a beginner or an advanced learner, DeepLearning.AI caters to all skill levels, making complex concepts easy to grasp. Gain practical knowledge, work on real-world projects, and join a thriving community of AI enthusiasts. Elevate your skills, boost your career, and unlock endless possibilities in AI. Embrace the future of technology with DeepLearning.AI today.

--------------------------------------------------------------------------------

**


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

Aggregated feedback:
1. SEO Reviewer suggests adding relevant keywords, internal links, and utilizing meta tags for improved search engine visibility.
2. Legal Reviewer emphasizes the importance of ensuring factual accuracy, substantiating claims, and updating disclaimers for transparency and compliance.
3. Ethics Reviewer highlights the need for authentic testimonials, disclosure of conflicts of interest, and avoiding exaggerated or false claims for credibility.

Final suggestions:
1. Incorporate relevant keywords like "online AI courses" and "best deep learning platform" in the title and content for better search engine optimization.
2. Review and update any claims about the platform's offerings to ensure they are factually accurate and substantiated.
3. Clearly disclose any potential conflicts of interest, such as financial relationships, to maintain transparency.
4. Avoid m

## Get the summary

In [15]:
print(res.summary)

Title: Mastering AI: Unleashing the Potential of DeepLearning.AI

Delve into the world of artificial intelligence with DeepLearning.AI, a leading platform offering top-tier online AI courses. Created by renowned expert Andrew Ng, these courses cover deep learning, neural networks, and more. Whether you're a novice or an expert, DeepLearning.AI caters to all levels, simplifying complex concepts. Gain practical knowledge, work on real projects, and join a vibrant AI community. Elevate your skills, advance your career, and explore limitless possibilities in AI. Discover the future of technology with DeepLearning.AI today.
